# A quick guide of using msBayesImpute for imputing missing values in mass-spectrometry proteomics/metabolomics data

MsBayesImpute is a method that combines Bayesian factorization models and probablistic dropout models for imputing missing values in mass-spectrometry proteomics data, accounting for the non-randomness of missing data points in MS data.  
MsBayesImpute is built upon the Pyro, a universal probabilistic programming language (PPL) written in Python.  
The required Python packages for running msBayesImpute are: **numpy** and **pandas** (for reading and writing csv files). 

## Install package

In [1]:
pip install ../dist/msbayesimputepy-0.1.0-py3-none-any.whl

Processing /Users/jiaojiaohe/Desktop/Project2 - Imputation/Factorization/msbayesimputepy/dist/msbayesimputepy-0.1.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


## Import required packages

In [2]:
from msbayesimputepy.generation import gen_data, gen_prob_miss
from msbayesimputepy.core import msBayesImpute
import numpy as np
import pandas as pd

## Generate data with artificial missing values

- The complete data is generated by defining protein size (D), sample size (N) and latent factor (K) and protein intercept (feature_inter).
- Missing values are created by a probabilistic dropout distribution characterized by inflection and scale parameters.

In [3]:
# generate data
np.random.seed(202411)
D = 5000
N = 200
K = 10
feature_inter = 20
ratio = 0.1
generation = gen_data(n_features = D, n_samples = N, n_factors = K, alpha_col = feature_inter)
generate_data = generation["data"] 

In [4]:
# generate missings
simuData = gen_prob_miss(generate_data, rho = 17.5, zeta = 2, rho_sd = 2, zeta_sd = 1, model = "perFeature")
simuData["X_miss"]

,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9,sample_10,...,sample_191,sample_192,sample_193,sample_194,sample_195,sample_196,sample_197,sample_198,sample_199,sample_200
feature_1,NaN,NaN,NaN,NaN,NaN,NaN,17.30993,NaN,19.36536,NaN,...,NaN,23.28569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
feature_2,21.35771,18.15361,25.70704,16.94694,20.50456,24.48452,21.99810,22.63902,25.21135,21.02091,...,27.28163,26.39487,20.52996,NaN,NaN,24.08542,26.66449,20.79465,20.65267,NaN
feature_3,21.27044,NaN,NaN,15.46760,13.54686,25.59514,20.41242,NaN,22.90487,18.25608,...,24.31340,25.65968,NaN,19.79213,NaN,23.21405,22.11616,20.80227,22.55273,NaN
feature_4,NaN,NaN,20.70437,20.01081,23.54113,NaN,23.70579,NaN,NaN,20.37735,...,19.97506,21.75430,NaN,22.94145,21.23125,20.16064,NaN,23.32555,NaN,NaN
feature_5,19.24773,19.72803,24.08629,NaN,NaN,26.66391,20.33935,17.18838,24.08511,23.71116,...,25.31210,24.71970,19.46055,27.40503,NaN,26.30030,25.52853,20.54043,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
feature_4996,NaN,NaN,19.60327,21.05771,19.93693,22.27992,20.48328,20.83228,20.71718,19.82733,...,23.72726,22.78724,NaN,19.25925,18.15226,NaN,NaN,NaN,NaN,NaN
feature_4997,NaN,22.32774,25.81730,19.30176,NaN,24.40884,21.64394,20.78337,23.04999,NaN,...,NaN,NaN,19.71806,26.14963,21.62542,21.28454,21.17954,24.10425,21.73265,NaN
feature_4998,22.44802,21.05945,20.06088,22.95352,21.75321,NaN,20.05362,NaN,22.30025,25.25971,...,NaN,NaN,NaN,23.82556,25.43966,23.65458,NaN,26.92724,NaN,NaN
feature_4999,NaN,22.63368,24.66836,22.98442,17.88588,24.81728,16.86489,23.40688,19.94300,18.62826,...,26.30401,18.09135,26.18410,30.52415,22.66261,NaN,23.26888,16.76908,19.38203,21.79644


## Read example dataset

Prior to running the model, certain data preprocessing steps are required: 
1. Remove completely missing proteins/metabolites
2. Ensure the dataset follows a normal distribution, such as by applying a log transformation to the raw data.

In [5]:
#read in data as dataframe
df = pd.read_csv('../data/hela_proteomics_example.csv', delimiter = ",", index_col = 0)
df

,X2019_12_18_14_35_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_19_19_48_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_20_14_15_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_27_12_29_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_29_15_06_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_29_18_18_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_02_17_38_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_11_17_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_16_58_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_20_10_Q.Exactive.HF.X.Orbitrap_6070,...,X2020_05_20_12_33_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_20_15_35_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_22_14_57_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_22_17_43_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_26_14_20_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_27_13_57_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_28_04_06_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_01_10_22_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_01_15_41_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_02_09_41_Q.Exactive.HF.X.Orbitrap_6070
AAAS,28.349295,27.657378,28.352160,26.825537,27.403650,27.891281,25.498262,27.351866,27.619749,27.299813,...,27.395910,27.721233,27.807533,28.051495,27.324573,29.119440,30.080299,27.298153,27.121097,29.037870
AACS,26.133163,25.018649,23.740468,NaN,26.948488,26.481022,NaN,NaN,25.623773,NaN,...,25.551510,24.915976,24.713925,25.608344,26.800453,27.061470,27.372936,NaN,NaN,25.989127
AAMDC,NaN,24.236226,NaN,NaN,23.864386,26.347547,NaN,24.433071,23.520373,25.660360,...,NaN,24.124709,NaN,26.030317,25.518603,25.989062,26.759769,23.002336,NaN,25.120167
AAMP,26.776933,26.270706,27.097882,26.256343,26.981635,27.849418,NaN,25.275225,27.135553,27.732792,...,25.740597,26.078081,25.856998,26.375159,25.204077,27.998626,27.600879,28.318466,27.968017,26.943690
AAR2,27.247805,27.210668,27.377411,NaN,26.519788,26.916955,NaN,24.845894,25.971277,26.896545,...,25.893215,26.726374,25.692774,25.536411,27.262844,28.294433,28.022898,NaN,26.707146,28.087955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWILCH,NaN,NaN,NaN,NaN,26.406378,26.037330,NaN,NaN,25.069482,24.685280,...,NaN,NaN,NaN,22.678440,23.852529,24.652394,26.042078,NaN,25.081966,NaN
ZWINT,24.379346,NaN,25.802919,24.687685,NaN,25.002430,NaN,NaN,NaN,24.662752,...,25.492406,26.087399,NaN,25.441676,26.853885,27.950714,28.031863,NaN,NaN,24.793970
ZYX,29.427124,28.080503,29.525072,28.783224,30.422219,30.928183,28.465093,29.726472,30.952062,31.004840,...,28.276308,28.691057,28.632182,28.992129,29.961546,30.793508,31.340482,29.382862,30.146980,29.846835
hCG_2014768.TMA7,28.079689,NaN,NaN,NaN,NaN,NaN,28.902336,28.933427,NaN,NaN,...,28.252144,NaN,26.563560,NaN,NaN,NaN,NaN,29.336370,29.706570,29.212339


## Perform imputation

- msBayesImpute is user-friendly because it requires no parameter tunning.
- In some cases, users would like to fix the number of latent factors or minimum explained variance, which can be set in msBayesImpute arguments by **n_components** (default is None) and **drop_factor_threshold** (default is 0.01).
- There is also two options in **convergence_mode**: "fast" and "slow" (defualt)

In [6]:
# perform model training
msBayes_model = msBayesImpute()  # arguments
msBayes_model.train(df)
impute_data = msBayes_model.predict(df)

Model option: alternating_featureWise, convergence mode: fast, shinrakge: HorseShoe

1. Initialize dropout curve:  rho - 24.20, zeta - 1.71
2. Initialize training and optimize number of factors.
  - Current factor number is: 25
  Epoch 1: elapsed time=00:01, LOSS=4.341978
  Epoch 100: elapsed time=00:06, deltaLOSS=0.008010 (0.18447%)
  Epoch 200: elapsed time=00:11, deltaLOSS=0.004017 (0.09252%)
  Epoch 300: elapsed time=00:17, deltaLOSS=0.006029 (0.13885%)
  Epoch 400: elapsed time=00:23, deltaLOSS=0.001822 (0.04196%)
  Epoch 500: elapsed time=00:28, deltaLOSS=-0.002411 (0.05553%)
  Epoch 600: elapsed time=00:34, deltaLOSS=0.003870 (0.08912%)
  Epoch 700: elapsed time=00:41, deltaLOSS=0.002123 (0.04889%)
  - Current factor number is: 3
  The final number is 3.
3. Start final model training.
  Step 1, refine factorization model.
  Epoch 1: elapsed time=00:50, LOSS=1.325378
  Step 2, refine feature-wise dropout curves.
  Epoch 100: elapsed time=00:56, deltaLOSS=0.002489 (0.67355%)
  Epo

In [7]:
#save the results to a csv file
impute_data.to_csv('imputation_python.csv')